# Declaration of Originality

![image.png](attachment:image.png)

**TEMASEK POLYTECHNIC**
<br>**SCHOOL OF INFORMATICS & IT**
<br>**DIPLOMA IN INFORMATION TECHNOLOGY**
<br>**MACHINE LEARNING FOR DEVELOPERS (CAI2C08)**
<br>**AY2025/2026 OCTOBER SEMESTER**

**PROJECT PROGRAM CODES**
* Student Name (Matric Number)  : Lim Cheng Xun (2401755J)
* Tutorial Group                : PC03
* Tutor						    : Mr Ruchir Srivastava
* Submission Date               : 11th February 2026


**Declaration of Originality**
* I am the originator of this work and I have appropriately acknowledged all other original sources used as my references for this work.
* I understand that Plagiarism is the act of taking and using the whole or any part of another person’s work, including work generated by AI, and presenting it as my own.
* I understand that Plagiarism is an academic offence and if I am found to have committed or abetted the offence of plagiarism in relation to this submitted work, disciplinary action will be enforced.

# Libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

from sklearn.cluster import KMeans
from sklearn import datasets

# 1. Business Understanding
Goal: The goal of my model it to predict whether annual income of an individual will exceeds $50K/yr based on census data collected.

# 2. Data Understanding

## 2.1 Load dataset

In [2]:
# This is the csv file path for the Adult Income Dataset
FILE_PATH = "adult.csv"

# Load dataset into a pandas DataFrame and replace ' ?' with NaN to ensure proper handling of missing values otherwise they may be misinterpreted as valid entries.
# Since the ' ?' entries are under the datatype object (string) they are considered proper entries as far as pandas is concerned.
# Thus, we specify na_values=' ?' in the read_csv function to convert these entries to NaN.
df = pd.read_csv(FILE_PATH, na_values=' ?', skipinitialspace=True)
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## 2.2 Summary Statistics

In [3]:
# Understand the type of variable for each column
# This will help in deciding which columns need to be encoded or transformed later on
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

In [4]:
# Check for missing data
# Here we check the number of missing values in each column
# The missing values are represented as NaN in the DataFrame
df.replace('?', np.nan, inplace=True)
df.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64

In [5]:
# Describe data distribution
df.describe()

# From the descriptive statistics,

# We can observe that the average age of individuals in the dataset is approximately 38 years, with a minimum age of 17 and a maximum age of 90.

# The average hours worked per week is around 40 hours, with some individuals working as few as 1 hour and others working up to 99 hours per week. (potential outliers)

# The capital gain and capital loss columns at the minimum, 25th percentile, 50th percentile (median), and 75th percentile all have a value of 0, 
# which indicates that a significant portion of individuals in the dataset do not have any capital gains or losses.

# The education-num column, which represents the number of years of education completed, has an average value of approximately 10 years,
# with a minimum of 1 year and a maximum of 16 years. This suggests that the dataset includes individuals with varying levels of education, 
# from those who have completed only a few years of schooling to those who have completed higher education.

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


### Target Variable (y)
y = df[income]

### Features Used (X)
X = df[age, workclass, fnlwgt, education, enducation-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, native-country]